In [2]:
import io
import pandas as pd
from pandas import json_normalize

import numpy as np
import json
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False,nb_workers=2)

import gspread as gs
from datetime import datetime

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [3]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

cashbac_key='Cashbac-GCP-Keys.json'
horego_key='Horego-GCP-Keys.json'

credentials = service_account.Credentials.from_service_account_file('Cashbac-GCP-Keys.json')
project_id = 'cashbac-31433'

# Make clients.
cashbac_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery

credentials = service_account.Credentials.from_service_account_file('Horego-GCP-Keys.json')
project_id = 'horego-big-query'

# Make clients.
horego_bqclient = bigquery.Client(
    credentials=credentials,
    project=project_id,
)

import psycopg2.pool
def db_connect_cashbac_nopool(host,database,user,password):
    """
    :return:
    """
    connection = psycopg2.connect(host=host,
                              database=database,
                              user=user,
                              password=password,
                              port=5432)
    return connection

In [8]:
query_string = f"""

    select * 
    from `horego-big-query.horego_datalake_dev.vw_outcrapper_google_results`
    where True
    and lower(kabupaten) like '%surabaya%'

"""

query_result = list(horego_bqclient.query(query_string).result(timeout=None))
temp_df = pd.DataFrame(data=[list(x.values()) for x in tqdm(query_result)], columns=list(query_result[0].keys()))
temp_df=temp_df.drop_duplicates()
temp_df.to_excel(f"results/Surabaya.xlsx",index=False, engine='xlsxwriter')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12924/12924 [00:00<00:00, 30481.92it/s]


In [10]:
import gspread as gs
gc = gs.service_account(filename=horego_key)
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1H9YtIjyPaW6hVSWbGh65KU0f9Q97QgoALp-nBsBvo3g/edit#gid=79371918')

worksheet = sheet.worksheet('Excluded Keywords')
name_exclude = pd.DataFrame(worksheet.get_all_values())
name_exclude.columns=name_exclude.iloc[0]
name_exclude=name_exclude[1:]
# name_exclude=name_exclude[['Exluded "Contains One of" Words (All Jakarta)']]
name_exclude.columns=['KEYWORD']
name_exclude=name_exclude[name_exclude.KEYWORD!='']
name_exclude.KEYWORD=name_exclude.KEYWORD.str.lower()